In [ ]:
import sys
import os
import time
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim.lr_scheduler import CyclicLR

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import math
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm_notebook
from pathlib import Path
import random
import torch.nn.init as init
import openpyxl

from datetime import timedelta

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

# import os

# google_drive_path = "G:/"

In [ ]:
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
else:
    print(gpu_info)

CUDA = torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")

# torch and python versions ====================================================
print('torch.__version__:',torch.__version__)
print('sys.version:',sys.version)

In [ ]:
DATASET_NAMES = ['Site3']
VARIABLE_NAMES = ['Temperature', 'V', 'Sin(dir)','Cos(dir)']
UNITS_OF_MEASUREMENT = ['°C', 'm/s', '', '']
POSITIVE_VARIABLES = [1]

In [ ]:
# Hyperparameters ==============================================================
LOOK_FORWARD_DAYS_LIST = [1,2,3]
DATASETS_TO_PROCESS = [0]
VARIABLES_TO_PROCESS = [1,2,3]

LOOK_BACK_DAYS = 3
TEST_FRACTION = 0.2
VALIDATION_FRACTION = 0.2
PATIENCE = 10
SHUFFLE = False
FORCE_TRAINING = False
DEBUG = False
USE_ADAM = True
USE_EARLY_STOP = True
USE_VALIDATION = True
DAYS_TO_PLOT = 3
POST_PROCESS = True
PRINT_RMSE = True

STANDARD_SCALERS = False
PLOT_DISTRIBUTIONS = False
OUTLIERS = 0.25
# ==============================================================================

if not USE_VALIDATION:
  VALIDATION_FRACTION = 0.0

HP = {0: # Site 3
         {0:  { 'HIDDEN_DIM' : 128,
                'N_LAYERS' : 2,
                'BATCH_SIZE' : 512,
                'LEARNING_RATE' : 0.001,
                'TEST_PLOT_START' : [9035, 8920, 8800],
                'EPOCHS' : 200},

          1:  { 'HIDDEN_DIM' :128,
                'N_LAYERS' : 2,
                'BATCH_SIZE' : 512,
                'LEARNING_RATE' : 0.001,
                'TEST_PLOT_START' : [9035, 8920, 8800],
                'EPOCHS' : 200},

          2:  { 'HIDDEN_DIM' : 128,
                'N_LAYERS' : 2, #4
                'BATCH_SIZE' : 512, #256
                'LEARNING_RATE' : 0.001,  #0.0001
                'TEST_PLOT_START' : [9035, 8920, 8800],
                'EPOCHS' : 200},

          3:  { 'HIDDEN_DIM' : 128,
                'N_LAYERS' : 2, #4
                'BATCH_SIZE' : 512, #256
                'LEARNING_RATE' : 0.001,  #0.0001
                'TEST_PLOT_START' : [9035, 8920, 8800],
                'EPOCHS' : 200}

        }
}

In [ ]:
# Loop through each dataset in DATASETS_TO_PROCESS
for dataset_id in DATASETS_TO_PROCESS:
  # Access the configuration dictionary for the current dataset
  dataset_config = HP[dataset_id]

  # Loop through each variable configuration within the dataset
  for variable_id in VARIABLES_TO_PROCESS:
    # Access the 'HIDDEN_DIM' value for the current variable
    hidden_dim = dataset_config[variable_id]['HIDDEN_DIM']
  for variable_id in VARIABLES_TO_PROCESS:
    # Access the 'N_LAYERS' value for the current variable
    n_layers = dataset_config[variable_id]['N_LAYERS']

In [ ]:
class Model(nn.Module):
    def __init__(self, input_dim, batch_size, learning_rate, hidden_dim, output_dim, n_layers, positive_variable, drop_prob=0.15):
        super(Model, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.positive_variable=positive_variable

        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.dropout = nn.Dropout(p=0.1)

        #self.bat_a = nn.BatchNorm1d(hidden_dim)
        #self.bat_b = nn.BatchNorm1d(hidden_dim)
        self.activation_a = nn.ReLU()
        #self.activation_b = nn.ReLU()
        self.fc_a = nn.Linear(hidden_dim, hidden_dim//2)
        self.fc_b = nn.Linear(hidden_dim//2, output_dim)

    def forward(self, x, h):
        out, h = self.gru(x, h)
        # print('out.shape (before):',out.shape)

        # The following statement is important: we are implementing a S2V model,
        # so we select the latest output.

        out=out[:,-1]
        # print('out.shape (after):',out.shape)


        out=self.fc_a(out)
        out=self.activation_a(out)
        out = self.dropout(out)
        out=self.fc_b(out)
        if self.positive_variable:
          #out=torch.exp(out)
          #out=torch.abs(out)
          #out=torch.square(out)
          pass

        return out, h

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

In [ ]:
def process_epoch(model, data_loader, criterion, optimizer, train, variable):
    n_layers = model.n_layers
    hidden_dim = model.hidden_dim
    batch_size = data_loader.batch_size

    if SHUFFLE:
        h = torch.zeros([n_layers, batch_size, hidden_dim]).to(device)
    else:
        h = model.init_hidden(batch_size)

    total_loss = 0.
    counter = 0

    for x, y, local_means in data_loader:
        counter += 1

        if SHUFFLE:
            h = torch.zeros([n_layers, batch_size, hidden_dim]).to(device)
        else:
            h = h.data

        model.zero_grad()


        y_hat, h = model(x.to(device).float(), h)
        base_loss = criterion(y_hat, y.to(device).float())

        #local_mean_loss = torch.mean(torch.abs(y_hat - local_means.to(device).float()))  # l1
        #local_mean_loss = torch.mean((y_hat - local_means.to(device).float()) ** 2) # mse
        local_mean_loss = criterion(y_hat, local_means.to(device).float())

        α = 0.5
        loss = α * base_loss + (1-α) * local_mean_loss
        #loss = local_mean_loss
        #loss = base_loss

        total_loss += loss.item()

        if train:
            loss.backward()
            optimizer.step()

        # if counter%20==0:
        #     print("Step: {}/{}, Average Loss: {}, Local Mean Loss: {}".format(
        #         counter, len(data_loader), total_loss/counter, local_mean_loss.item()))

        if counter%20==0:
            print("Step: {}/{}, Average Loss: {}".format(
                counter, len(data_loader), total_loss/counter))


    avg_loss = total_loss/len(data_loader)
    return avg_loss

def train(model_name, train_loader, validation_loader, learning_rate, batch_size, positive_variable=False,
          hidden_dim = hidden_dim, n_layers = n_layers, patience=None, epochs=None):
    input_dim = next(iter(train_loader))[0].shape[2]
    output_dim = 1

    print('Input dim.:', input_dim)
    print('Output dim.:', output_dim)

    # The model
    model = Model(input_dim, hidden_dim=hidden_dim, output_dim=1,batch_size=batch_size, learning_rate=learning_rate, n_layers=n_layers, positive_variable=positive_variable)
    model.to(device)

    # Loss function and optimizer
    #criterion = nn.MSELoss()
    criterion = nn.L1Loss()

    if USE_ADAM:
      optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    else:
      optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

    model.train()

    epoch=1
    patience_counter = 0
    min_validation_loss=None
    validation_loss=None

    train_loss_list=[]
    validation_loss_list=[]

    # Training loop ============================================================
    while True:
        print("Epoch {}".format(epoch))

        print('Training...')
        train_loss = process_epoch(model, train_loader, criterion, optimizer, train = True, variable=variable)

        if USE_VALIDATION:
          print('Validation...')
          validation_loss = process_epoch(model, validation_loader, criterion, optimizer, train = False, variable=variable)
          validation_loss_list.append(validation_loss)

        train_loss_list.append(train_loss)

        if min_validation_loss is None or validation_loss<min_validation_loss or not USE_VALIDATION:
          # Save the model
          print('Saving the model...')
          torch.save(model, model_name)

          min_validation_loss = validation_loss
          patience_counter = 0
        else:
          print('Increasing patience counter')
          patience_counter +=1

        print("Epoch {} Completed, Train Loss: {}, Validation Loss: {}, Patience counter: {}".format(epoch, train_loss, validation_loss, patience_counter))

        if (patience_counter>patience and USE_EARLY_STOP) or epoch==epochs:
          break

        epoch += 1

    print('Restoring the best model...')
    model = torch.load(model_name, weights_only=False)
    return model, train_loss_list, validation_loss_list

In [ ]:
def evaluate(model, test_x, test_y, label_scaler, post_process, mm5_array):
    outputs = []
    targets = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for i in range(test_x.shape[0]):
        inp = torch.from_numpy(np.expand_dims(test_x[i], axis=0))

        out, h = model(inp.to(device).float(), None)

        outputs.append(out[0,0].item())
        targets.append(test_y[i])

        if i%5000==0:
            print(i,'/',test_x.shape[0],sep='')

    predictions_array = label_scaler.inverse_transform(np.array(outputs).reshape(-1, 1))
    targets_array = label_scaler.inverse_transform(np.array(targets).reshape(-1, 1))

    if post_process:
      predictions_array = predictions_array * (mm5_array!=0)

    return predictions_array, targets_array

In [ ]:
def se_array(x, y):
  return np.square(x-y)

def ae_array(x, y):
  return np.abs(x - y)

def rmse(x,y):
  return np.sqrt(mean_squared_error(x,y))

def mae(x, y):
    return np.mean(np.abs(x-y))

def latex_row(a,b,c,bigger):
  if a>b and bigger or a<b and not bigger:
    return '\\textbf{'+str(round(a, 3))+'}\\\\ '+str(round(b, 3))+'\\\\ '+str(round(c, 2))+'\%'
  else:
    return str(round(a, 3))+'\\\\ \\textbf{'+str(round(b, 3))+'}\\\\ '+str(round(c, 2))+'\%'

In [ ]:
def calculate_local_mean(values, window_size=3):
    result = np.zeros_like(values)
    padded = np.pad(values, (window_size, window_size), mode='edge')

    for i in range(len(values)):
        window_start = i
        window_end = i + 2 * window_size + 1
        result[i] = np.mean(padded[window_start:window_end])

    return result

In [ ]:
for LOOK_FORWARD_DAYS in LOOK_FORWARD_DAYS_LIST:
  for dataset in DATASETS_TO_PROCESS:
      for variable in [2,3]:  # VARIABLES_TO_PROCESS:
        print('='*80)
        print('Look Forward Days:', LOOK_FORWARD_DAYS)
        print('Dataset:', DATASET_NAMES[dataset])
        print('Variable:', VARIABLE_NAMES[variable])
        print('_'*80)

        if variable not in HP[dataset].keys():
          continue

        data_root = '/content/drive/MyDrive/Colab Notebooks/datasets/energy/'
        root = data_root + 'results/'

        suffix = '_look_forward_days_' + str(LOOK_FORWARD_DAYS) + '_dataset_' + str(dataset) + '_variable_' + str(variable)

        report_name = root + 'report'+ suffix + '.txt'
        report_file = open(report_name, "w")

        report_file.write('Dataset {}, Variable {}\n'.format(str(dataset), VARIABLE_NAMES[variable]))
        report_file.write('_'*80+'\n')

        # Create dataframe =======================================================

        # ============================================================================
        # DATA LOADING
        # ============================================================================
        # NOTE: Due to confidentiality agreements, the datasets cannot be publicly
        # shared. The code is provided for transparency and reproducibility of the
        # methodology. Researchers with similar datasets can adapt this code for
        # their own data.
        #
        # Expected file structure in Google Drive:
        # MyDrive/Colab Notebooks/datasets/energy/dataset_{0,1}_look_forward_days_{1,2,3}.xlsx
        # ============================================================================

        url = data_root + 'dataset_'+str(dataset)+'_look_forward_days_' + str(LOOK_FORWARD_DAYS)+'.xlsx'
        df = pd.read_excel(url)

        pd.set_option('display.float_format', lambda x: '%.2f' % x)
        #print(df.head)

        df['dayofyear'] = df.apply(lambda x: x['Data_R'].dayofyear,axis=1)
        df['hour'] = df.apply(lambda x: x['Ora_R'].hour,axis=1)
        #df['dayofweek'] = df.apply(lambda x: x['Data_R'].dayofweek,axis=1)
        #df['month'] = df.apply(lambda x: x['Data_R'].month,axis=1)
        #df['minute'] = df.apply(lambda x: x['Ora_R'].minute,axis=1)
        #df['DateTime'] = pd.to_datetime(df['Data_R'].astype(str) + ' ' + df['Ora_R'].astype(str))

        datetime_info = df[['Data_R', 'Ora_R']].copy()

        df = df.drop(["Data_R", "Ora_R", "Data_MM5", "Ora_MM5"], axis=1)


        df = df.replace("None", np.nan)

        if PLOT_DISTRIBUTIONS:
            df[df['GHI_R'].notna()]['GHI_R'].plot.hist(bins=100)
            plt.pause(0.001)
            df[df['Temperatura_R'].notna()]['Temperatura_R'].plot.hist(bins=100)
            plt.pause(0.001)
            df[df['V_Tot_R [30m]'].notna()]['V_R'].plot.hist(bins=100)
            plt.pause(0.001)
            df[df['Sin_R'].notna()]['Sin_R'].plot.hist(bins=100)
            plt.pause(0.001)
            df[df['Cos_R'].notna()]['Cos_R'].plot.hist(bins=100)
            plt.pause(0.001)
            break

        NUMBER_OF_VARIABLES = (len(df.columns)-2)//2
        print('Number of variables:', NUMBER_OF_VARIABLES)

        # Select current hyperparameters =========================================
        HIDDEN_DIM = HP[dataset][variable]['HIDDEN_DIM']
        N_LAYERS = HP[dataset][variable]['N_LAYERS']
        BATCH_SIZE = HP[dataset][variable]['BATCH_SIZE']
        LEARNING_RATE = HP[dataset][variable]['LEARNING_RATE']
        TEST_PLOT_START = HP[dataset][variable]['TEST_PLOT_START'][LOOK_FORWARD_DAYS-1]
        TEST_PLOT_STOP = TEST_PLOT_START+144*DAYS_TO_PLOT
        EPOCHS = HP[dataset][variable]['EPOCHS']

        for hp in HP[dataset][variable].keys():
            print(hp,': ', HP[dataset][variable][hp],sep='')
            pass

        # Create scalers =========================================================
        data = df.select_dtypes(include=['number', 'float', 'int']).astype('float32').to_numpy()
        print(type(data))

        if STANDARD_SCALERS:
          scaler = StandardScaler()
          label_scaler = StandardScaler()
          mm5_scaler = StandardScaler()
        else:
          scaler = MinMaxScaler()
          label_scaler = MinMaxScaler()
          mm5_scaler = MinMaxScaler()

        if dataset==0 or variable==0:
          label_scaler.fit(data[:,variable].reshape(-1,1))
          mm5_scaler.fit(data[:,variable+NUMBER_OF_VARIABLES].reshape(-1,1))

        else:
          label_scaler.fit(data[:,variable].reshape(-1,1))
          mm5_scaler.fit(data[:,variable+NUMBER_OF_VARIABLES].reshape(-1,1))

        data = scaler.fit_transform(data)

        if DEBUG:
            print(scaler.data_min_)
            print(scaler.data_max_)
            print(label_scaler.data_min_)
            print(label_scaler.data_max_)
            print(mm5_scaler.data_min_)
            print(mm5_scaler.data_max_)


        # Define lookback period and split inputs/labels =========================
        look_back_steps = 144 * LOOK_BACK_DAYS
        look_forward_steps = 144 * LOOK_FORWARD_DAYS

        inputs = np.zeros((len(data)-look_back_steps-look_forward_steps+1,look_back_steps,NUMBER_OF_VARIABLES+2))
        local_means = np.zeros(len(data)-look_back_steps-look_forward_steps+1)
        labels = np.zeros(len(data)-look_back_steps-look_forward_steps+1)
        dates = np.zeros(len(data)-look_back_steps-look_forward_steps+1, dtype='object')
        hours = np.zeros(len(data)-look_back_steps-look_forward_steps+1, dtype='object')

        if DEBUG:
            print(data.shape)
            print(inputs.shape)
            print(labels.shape)

        mask=np.full(inputs.shape[0], True)

        # ONLY MM5 DATA:

        # The input is a sequence of vectors containing the MM5 values from time
        # 'T-look_back_steps+1' to time 'T' (where T is future instant of time the
        # prediction is referred to)
        # Try look_back_steps=look_forward_steps=1

        # The models takes in input only MM5 values that cannot be null
        # That's why some data rows previously deleted now are included and the
        # MM5 performance metrics are slightly different.

        for i in range(look_back_steps, len(data) - look_forward_steps + 1):

            index = i-look_back_steps

            inputs[index] = data[index : i, NUMBER_OF_VARIABLES:] # MM5
            # inputs[index] = data[index : i]

            labels[index] = data[i + look_forward_steps - 1, variable]

            dates[index] = datetime_info['Data_R'][i + look_forward_steps - 1]  # Inputs and labels are NumPy because of the scaler. datetime_info, on the other hand, is a pandas DataFrame, which has specific, different indexing methods.
            hours[index] = datetime_info['Ora_R'][i + look_forward_steps - 1]

            if np.isnan(inputs[index]).any() or np.isnan(labels[index]).any():
                #print('input:',inputs[index])
                #print('label:',labels[index])
                mask[index] = False

            if i%10000==0 and DEBUG:
                print(i)
                print('-'*80)
                print(inputs[index])
                print(labels[index])

        # Removing tuples containing nan values
        if DEBUG or True:
            print('BEFORE')
            print(inputs.shape)
            print(labels.shape)
            print('nan inputs:', np.isnan(inputs).any())
            print('nan labels:', np.isnan(labels).any())
            print('Number of nan tuples:', np.sum(np.logical_not(mask)))

        inputs=inputs[mask]
        labels=labels[mask]
        dates = dates[mask]
        hours = hours[mask]

        if DEBUG or True:
            print('AFTER')
            print(inputs.shape)
            print(labels.shape)
            print('nan inputs:', np.isnan(inputs).any())
            print('nan labels:', np.isnan(labels).any())

        # inputs = inputs.reshape(-1, look_back_steps, df.shape[1])
        inputs = inputs.reshape(-1, look_back_steps, NUMBER_OF_VARIABLES +2) # MM5
        labels = labels.reshape(-1, 1)

        local_means = calculate_local_mean(labels.flatten())
        local_means = local_means.reshape(-1, 1)

        # Split data into training set, validation set and test set
        validation_portion = int(VALIDATION_FRACTION*len(inputs))
        test_portion = int(TEST_FRACTION*len(inputs))
        validation_test_portion = validation_portion + test_portion

        train_x = inputs[:-validation_test_portion]
        train_y = labels[:-validation_test_portion]
        train_local_means = local_means[:-validation_test_portion]

        validation_x = inputs[-validation_test_portion:-test_portion]
        validation_y = labels[-validation_test_portion:-test_portion]
        validation_local_means = local_means[-validation_test_portion:-test_portion]

        test_x = inputs[-test_portion:]
        test_y = labels[-test_portion:]
        test_dates = dates[-test_portion:]
        test_hours = hours[-test_portion:]


        # Create dataloaders =====================================================
        train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y), torch.from_numpy(train_local_means))
        train_loader = DataLoader(train_data, shuffle=SHUFFLE, batch_size=BATCH_SIZE, drop_last=True)

        validation_data = TensorDataset(torch.from_numpy(validation_x), torch.from_numpy(validation_y), torch.from_numpy(validation_local_means))
        validation_loader = DataLoader(validation_data, shuffle=SHUFFLE, batch_size=BATCH_SIZE, drop_last=True)

        train_loss_list=None
        validation_loss_list=None

        # Create the model =======================================================
        model_name = root + 'model' + suffix + '.zip'
        print(model_name)

        if Path(model_name).is_file() and not FORCE_TRAINING:
            model = torch.load(model_name, weights_only=False)

        else:
            model, train_loss_list, validation_loss_list = train(model_name, train_loader, validation_loader, batch_size=BATCH_SIZE, learning_rate=LEARNING_RATE, hidden_dim=HIDDEN_DIM, n_layers=N_LAYERS, positive_variable=(variable in POSITIVE_VARIABLES), patience=PATIENCE, epochs=EPOCHS)

        model.eval()

        # Loss graphs ============================================================
        if train_loss_list is not None and validation_loss_list is not None:
          plt.figure(figsize=(15,10))
          plt.subplot(2,2,1)
          ma_domain=[x for x in range(2,len(validation_loss_list))]
          plt.plot(train_loss_list, color="navajowhite", label="Train Loss", linewidth=0.7)
          plt.plot(validation_loss_list, color="lightblue", label="Validation Loss", linestyle="dashed", linewidth=0.7)

          train_loss_ma=[(train_loss_list[i]+train_loss_list[i+1]+train_loss_list[i+2])/3 for i in range(len(train_loss_list)-2)]
          validation_loss_ma=[(validation_loss_list[i]+validation_loss_list[i+1]+validation_loss_list[i+2])/3 for i in range(len(validation_loss_list)-2)]

          plt.plot(ma_domain, train_loss_ma, color="darkorange", label="Train Loss (MA)", linewidth=0.7)
          plt.plot(ma_domain, validation_loss_ma, color="blue", label="Validation Loss (MA)", linestyle="dashed", linewidth=0.7)

          plt.axvline(x = validation_loss_ma.index(min(validation_loss_ma)), color="r", linewidth=0.7)

          plt.ylabel('Loss')
          plt.xlabel('Epoch')
          plt.grid()
          plt.legend()

          # Save the figure ========================================================
          figure_name = root + 'loss_figure'+suffix+'.png'
          plt.savefig(figure_name, dpi=(250), bbox_inches='tight')

        if DEBUG:
            for name, param in model.named_parameters():
                if param.requires_grad:
                    print(name, param.data)
                    pass

        # Evaluation =============================================================

        if dataset==0 or variable==0:
          #mm5_array = mm5_scaler.inverse_transform(test_x[:, -1, variable + NUMBER_OF_VARIABLES].reshape(-1,1))
          mm5_array = mm5_scaler.inverse_transform(test_x[:, -1, variable].reshape(-1,1)) # MM5
        else:
          #mm5_array = mm5_scaler.inverse_transform(test_x[:, -1, variable + NUMBER_OF_VARIABLES].reshape(-1,1))
          mm5_array = mm5_scaler.inverse_transform(test_x[:, -1, variable].reshape(-1,1)) # MM5

        predictions_array, targets_array = evaluate(model, test_x, test_y, label_scaler, (variable in POSITIVE_VARIABLES) and POST_PROCESS, mm5_array)


        filename_xlsx = root + 'arrays'+suffix+'.xlsx'
        formatted_dates = [d.strftime('%d/%m/%Y') for d in test_dates]
        df_results = pd.DataFrame({
                      'Date': formatted_dates,
                      'Hour': test_hours,
                      'Real': targets_array.flatten(),
                      'MM5': mm5_array.flatten(),
                      'Prediction': predictions_array.flatten()
                      })

        df_results.to_excel(filename_xlsx, index=False)

        print('_'*80)

        # RMSE ===================================================================
        if PRINT_RMSE:
          prediction_rmse = rmse(predictions_array, targets_array)
          mm5_rmse = rmse(mm5_array, targets_array)
          delta_rmse = round((prediction_rmse-mm5_rmse)/mm5_rmse*100,2)

          print('Prediction rmse: {:.3f}'.format(prediction_rmse))
          print('MM5 rmse: {:.3f}'.format(mm5_rmse))
          print('Delta rmse: {:.3f} %'.format(delta_rmse))

          report_file.write('Prediction rmse: {:.3f}\n'.format(prediction_rmse))
          report_file.write('MM5 rmse: {:.3f}\n'.format(mm5_rmse))
          report_file.write('Delta rmse: {:.3f} %\n\n'.format(delta_rmse))

          print('_'*80)

        # MAE ===================================================================
        prediction_mae = mae(predictions_array, targets_array)
        mm5_mae = mae(mm5_array, targets_array)
        delta_mae = round((prediction_mae-mm5_mae)/mm5_mae*100,2)*np.sign(mm5_mae)

        print('MM5 mae: {:.3f}'.format(mm5_mae))
        print('Prediction mae: {:.3f}'.format(prediction_mae))
        print('Delta mae: {:.3f} %'.format(delta_mae))

        report_file.write('MM5 mae: {:.3f}\n'.format(mm5_mae))
        report_file.write('Prediction mae: {:.3f}\n'.format(prediction_mae))
        report_file.write('Delta mae: {:.3f} %\n\n'.format(delta_mae))

        print('_'*80)

        number_of_outliers = int(len(targets_array)*OUTLIERS)
        print('number_of_outliers:',number_of_outliers)

        # MAE Outliers ===========================================================
        prediction_ae_array=ae_array(predictions_array, targets_array).T[0]
        mm5_ae_array=ae_array(mm5_array, targets_array).T[0]

        indices_of_prediction_outliers = np.argpartition(prediction_ae_array, -number_of_outliers)[-number_of_outliers:]
        indices_of_mm5_outliers = np.argpartition(mm5_ae_array, -number_of_outliers)[-number_of_outliers:]

        indices_of_outliers = list(set(indices_of_prediction_outliers) | set(indices_of_mm5_outliers))

        print(len(predictions_array))
        print(len(indices_of_outliers))

        outliers_prediction_mae = mae(predictions_array[indices_of_outliers], targets_array[indices_of_outliers])
        outliers_mm5_mae = mae(mm5_array[indices_of_outliers], targets_array[indices_of_outliers])
        outliers_delta_mae = round((outliers_prediction_mae-outliers_mm5_mae)/outliers_mm5_mae*100,2)*np.sign(outliers_mm5_mae)

        print('Outliers MM5 mae: {:.3f}'.format(outliers_mm5_mae))
        print('Outliers Prediction mae: {:.3f}'.format(outliers_prediction_mae))
        print('Outliers Delta mae: {:.3f} %'.format(outliers_delta_mae))

        report_file.write('Outliers MM5 mae: {:.5f}\n'.format(outliers_mm5_mae))
        report_file.write('Outliers Prediction mae: {:.3f}\n'.format(outliers_prediction_mae))
        report_file.write('Outliers Delta mae: {:.3f} %\n\n'.format(outliers_delta_mae))

        print('_'*80)


        # R^2 ====================================================================
        r2_prediction = r2_score(targets_array, predictions_array)
        r2_mm5 = r2_score(targets_array, mm5_array)
        delta_r2 = round((r2_prediction-r2_mm5)/r2_mm5*100,2)*np.sign(r2_mm5)

        print('MM5 R²: {:.3f}'.format(r2_mm5))
        print('Prediction R²: {:.3f}'.format(r2_prediction))
        print('Delta R²: {:.3f} %'.format(delta_r2))

        report_file.write('MM5 R²: {:.3f}\n'.format(r2_mm5))
        report_file.write('Prediction R²: {:.3f}\n'.format(r2_prediction))
        report_file.write('Delta R²: {:.3f} %\n\n'.format(delta_r2))

        print('_'*80)

        # R^2 Outliers ===========================================================
        prediction_se_array=se_array(predictions_array, targets_array).T[0]
        mm5_se_array=se_array(mm5_array, targets_array).T[0]

        indices_of_prediction_outliers = np.argpartition(prediction_se_array, -number_of_outliers)[-number_of_outliers:]
        indices_of_mm5_outliers = np.argpartition(mm5_se_array, -number_of_outliers)[-number_of_outliers:]

        indices_of_outliers = list(set(indices_of_prediction_outliers) | set(indices_of_mm5_outliers))

        print(len(predictions_array))
        print(len(indices_of_outliers))

        outliers_r2_prediction = r2_score(predictions_array[indices_of_outliers], targets_array[indices_of_outliers])
        outliers_r2_mm5 = r2_score(mm5_array[indices_of_outliers], targets_array[indices_of_outliers])
        outliers_delta_r2 = round((outliers_r2_prediction-outliers_r2_mm5)/outliers_r2_mm5*100,2)*np.sign(outliers_r2_mm5)

        print('Outliers MM5 R²: {:.3f}'.format(outliers_r2_mm5))
        print('Outliers Prediction R²: {:.3f}'.format(outliers_r2_prediction))
        print('Outliers Delta R²: {:.3f} %'.format(outliers_delta_r2))

        report_file.write('Outliers MM5 R²: {:.3f}\n'.format(outliers_r2_mm5))
        report_file.write('Outliers Prediction R²: {:.3f}\n'.format(outliers_r2_prediction))
        report_file.write('Outliers Delta R²: {:.3f} %\n\n'.format(outliers_delta_r2))

        print('_'*80)

        # LateX
        tabular='\\begin{tabular}[c]{@{}c@{}} '+\
        latex_row(mm5_mae,prediction_mae,delta_mae, False) +\
        '\end{tabular} & \\begin{tabular}[c]{@{}c@{}}'+\
        latex_row(r2_mm5,r2_prediction,delta_r2, True) +\
        '\end{tabular}'

        outliers_tabular='\\begin{tabular}[c]{@{}c@{}} '+\
        latex_row(outliers_mm5_mae,outliers_prediction_mae,outliers_delta_mae, False) +\
        '\end{tabular} & \\begin{tabular}[c]{@{}c@{}}'+\
        latex_row(outliers_r2_mm5,outliers_r2_prediction,outliers_delta_r2, True) +\
        '\end{tabular}'

        report_file.write(tabular+'\n\n')
        report_file.write(outliers_tabular)

        report_file.close()

        # Plot ===================================================================
        days=(TEST_PLOT_STOP-TEST_PLOT_START)//144
        print('Plotting results over ', days,' days',sep='')

        shift=520#(3-LOOK_FORWARD_DAYS)*260

        plt.figure(figsize=(20,10))
        plt.subplot(2,2,1)
        plt.plot(targets_array[TEST_PLOT_START+shift:TEST_PLOT_STOP+shift], color="g", label="Real", linewidth=0.7)
        plt.plot(mm5_array[TEST_PLOT_START+shift:TEST_PLOT_STOP+shift], color="b", label="MM5", linestyle="dashed", linewidth=0.7)
        plt.plot(predictions_array[TEST_PLOT_START+shift:TEST_PLOT_STOP+shift], color="r", label="Prediction", linestyle="dashdot", linewidth=0.7)
        plt.ylabel(VARIABLE_NAMES[variable]+' ['+UNITS_OF_MEASUREMENT[variable]+']')
        plt.xlabel('Time')
        plt.legend()

        # Save the figure ========================================================
        figure_name = root + 'figure'+suffix+'.png'
        plt.savefig(figure_name, dpi=(250), bbox_inches='tight')
        plt.show()
        if PLOT_DISTRIBUTIONS:
            break

In [ ]:
if not PLOT_DISTRIBUTIONS:
    # Plot =====================================================================
    TEST_PLOT_START = 5100#6710
    TEST_PLOT_STOP = TEST_PLOT_START + 144*3

    days=(TEST_PLOT_STOP-TEST_PLOT_START)//144
    print('Plotting results over ', days,' days',sep='')

    plt.figure(figsize=(20,10))
    plt.subplot(2,2,1)
    plt.plot(targets_array[TEST_PLOT_START:TEST_PLOT_STOP], color="g", label="Real", linewidth=0.7)
    plt.plot(mm5_array[TEST_PLOT_START:TEST_PLOT_STOP], color="b", label="MM5", linestyle='dashed', linewidth=0.7)
    plt.plot(predictions_array[TEST_PLOT_START:TEST_PLOT_STOP], color="r", label="Predicted", linestyle='dashdot', linewidth=0.7)
    plt.ylabel(VARIABLE_NAMES[variable])
    plt.legend()